In [46]:
import pandas as pd
from tqdm import tqdm
import re
import spacy
from tqdm import tqdm
from unidecode import unidecode
from dateutil import parser

nlp = spacy.load('en')

In [3]:
def name_disambiguation(name):
    if (name == "Khaleda" or name == "Zia" or name == "Begum Khaleda Zia"):
        return "Khaleda Zia"
    if (name == "Hasina" or name == "Sheikh"):
        return "Sheikh Hasina"
    if (name == "Fakhrul"):
        return "Mirza Fakhrul Islam Alamgir"
    if (name == "Muhith" or name == "AMA Muhith" or name == "MA Muhith"):
        return "Abul Maal Abdul Muhith"
    if (name == "Nizami" or name == "Motiur Rahman"):
        return "Motiur Rahman Nizami"
    if (name == "Modi"):
        return "Narendra Modi"
    if (name == "Bangabandhu" or name == "Sheikh Mujib" or name == "Sheikh Mujib" or name == "Bangabandhu Sheikh Mujibur" or name == "Sheikh Mujibur Rahman"):
        return "Bangabandhu Sheikh Mujibur Rahman"
    if (name == "Tarique"):
        return "Tarique Rahman"
    if (name == "Avijit"):
        return "Avijit Roy"
    if (name == "Mozena"):
        return "Dan Mozena"
    if (name == "Yunus" or name == "Mohammad Yunus"):
        return "Muhammad Yunus"

### Date limit: 2013-07-08 to 2016-06-14 on DT, New Age and Daily Sun

In [5]:
dt = pd.read_json('Data/DT/bd_news_dt.json')
newage = pd.read_pickle('Data/New Age/NewAge_ent.pkl')
dsun = pd.read_pickle('Data/Daily Sun/DailySun_ent_1.pkl')

In [14]:
dt.head()

,_id,is_negative,news_crawled_date,news_headline,news_image_urls,news_keywords,news_location,news_ml_tags,news_naive_tags,news_ner_tags,news_original_tags,news_publish_date,news_reporters,news_text,news_url,newspaper_name,newspaper_url
0,{'$oid': '575f79f55388c1567baebb2a'},True,{'$date': '2016-06-13T23:28:50.269Z'},Ashram residents remain in panic,[http://www.dhakatribune.com/sites/default/fil...,"[bangladesh, ashram, monastery, nityanando, mo...",Pabna,NaN,[crime],"{'persons': ['Hemayetpur Dham', 'Noni Gopal', ...",[crime],{'$date': '2016-06-14T01:28:00.000Z'},[[Md Emroz Khandakar]],People who live in Sri Sri Thakur Anukulchandr...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com
1,{'$oid': '575f79f85388c1567baebb2b'},True,{'$date': '2016-06-13T23:28:54.437Z'},Bangladeshi abducted from Bandarban border,[http://www.dhakatribune.com/sites/default/fil...,"[naikkangchhari, area, bandarban, woodcutters,...",Bandarban,NaN,[crime],"{'persons': ['Col Azim', 'Ula Mang Marma', 'He...",[bangladesh],{'$date': '2016-06-14T01:27:00.000Z'},[[S Bashu Das]],Separatists in Myanmar abducted a Bangladeshi ...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com
2,{'$oid': '575f79fc5388c1567baebb2c'},True,{'$date': '2016-06-13T23:28:57.408Z'},‘Narayanganj killing was pre-planned’,[http://www.dhakatribune.com/sites/default/fil...,"[murder, rab, picked, preplanned, vehicles, ki...",Narayanganj,NaN,[crime],"{'persons': ['Arif Hossain', 'MA Rana', 'Polas...",[crime],{'$date': '2016-06-14T01:25:00.000Z'},[[Tanveer Hossain]],Two personnel of the Rapid Action Battalion ye...,http://www.dhakatribune.com/crime/2016/jun/14/...,Dhaka Tribune,http://www.dhakatribune.com
3,{'$oid': '575f79fe5388c1567baebb2d'},True,{'$date': '2016-06-13T23:29:00.538Z'},Slum kids see no future without birth registra...,[http://www.dhakatribune.com/sites/default/fil...,"[dhaka, school, kids, pay, slum, shabnam, pall...",National,NaN,[accident],"{'persons': ['Shabnam', 'Shabnam', 'Shabnam'],...",[bangladesh],{'$date': '2016-06-14T01:24:00.000Z'},[Mohammad Jamil Khan],Thirteen-year-old Shabnam had big dreams for h...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com
4,{'$oid': '575f7a015388c1567baebb2e'},True,{'$date': '2016-06-13T23:29:03.326Z'},Mobile-based agro advisory service makes farme...,[http://www.dhakatribune.com/sites/default/fil...,"[advisory, life, grameenphone, service, farmer...",National,NaN,[accident],"{'persons': ['Hanif', 'Hanif', 'Hanif', 'Shefa...",[bangladesh],{'$date': '2016-06-14T01:22:00.000Z'},[Abu Bakar Siddique],Amela Begum in Rangpur believes she could have...,http://www.dhakatribune.com/bangladesh/2016/ju...,Dhaka Tribune,http://www.dhakatribune.com


In [18]:
# dt['news_publish_date'] = pd.to_datetime(dt['news_publish_date'])

In [51]:
def conv_to_datetime(date_dict):
    return parser.parse(list(date_dict.items())[0][1])

In [54]:
dt['news_publish_date'] = dt['news_publish_date'].apply(conv_to_datetime)

In [63]:
newage['date_published'] = pd.to_datetime(newage['date_published'])

In [66]:
dsun['date_published'] = pd.to_datetime(dsun['date_published'])